# Blue Brain BioExplorer

![](../bioexplorer_banner.png)

## Simplest script to create a visualization of an animated spike

In [ ]:
from bioexplorer import BioExplorer, MovieMaker, Protein, Sugars, Quaternion
uri = 'localhost:5000'
be = BioExplorer(uri)
core = be.core_api()
be.reset()
print(be.version())

In [ ]:
'''Resources'''
resource_folder = '../../tests/test_files/'
pdb_folder = resource_folder + 'pdb/'
complex_folder = resource_folder + 'pdb/glycans/complex/'

protein_representation = BioExplorer.REPRESENTATION_ATOMS_AND_STICKS
protein_radius_multiplier = 1.0
glycan_representation = BioExplorer.REPRESENTATION_ATOMS_AND_STICKS
glycan_radius_multiplier = 1.0

glycan_folder = pdb_folder + 'glycans/'
complex_paths = [glycan_folder + 'complex/33.pdb', glycan_folder + 'complex/34.pdb',
                 glycan_folder + 'complex/35.pdb',glycan_folder + 'complex/36.pdb']
high_mannose_paths = [glycan_folder + 'high-mannose/1.pdb', 
                      glycan_folder + 'high-mannose/2.pdb',
                      glycan_folder + 'high-mannose/3.pdb',
                      glycan_folder + 'high-mannose/4.pdb']
hybrid_paths = [glycan_folder + 'hybrid/24.pdb']
o_glycan_paths = [glycan_folder + 'o-glycan/12.pdb']

In [ ]:
'''Default side view'''
status = be.core_api().set_camera(
    orientation=[0.707, 0, -0.707, 0.0],
    position=[-26.348, 0.172, 0.620],
    target=[-2.728, 0.172, 0.620]
)

In [ ]:
def add_spike(orientation_randon_seed, open_spike=False):
    
    core.set_application_parameters(image_stream_fps=0)

    '''Protein'''
    source = pdb_folder + 'sars-cov-2-v1.pdb'
    if open_spike:
        source = pdb_folder + '6vyb.pdb'

    spike = Protein(
        sources=[source],
        load_hydrogen=False,
        load_non_polymer_chemicals=False,
    )

    name = be.NAME_PROTEIN_S_CLOSED
    if open_spike:
        name = be.NAME_PROTEIN_S_OPEN
        
    status = be.add_protein(
        name=name, protein=spike,
        atom_radius_multiplier=protein_radius_multiplier,
        representation=protein_representation,
    )
    
    '''High-mannose'''
    indices = [61, 122, 234, 603, 709, 717, 801, 1074]

    high_mannose_glycans = Sugars(
        rotation=Quaternion(0.707, 0.0, 0.0, 0.707),
        assembly_name=name, name=be.NAME_GLYCAN_HIGH_MANNOSE, 
        protein_name=name, source=high_mannose_paths[0], 
        site_indices=indices,
        representation=glycan_representation,
        atom_radius_multiplier=glycan_radius_multiplier,
        assembly_params=[0, 0, 0.0, orientation_randon_seed, 0.1]
    )
    status = be.add_glycans(high_mannose_glycans)
    
    '''O-Glycans'''
    protein_name = be.NAME_PROTEIN_S_CLOSED
    if open_spike:
        protein_name = be.NAME_PROTEIN_S_OPEN

    for index in [323, 325]:
        o_glycan_name = name + '_' + be.NAME_GLYCAN_O_GLYCAN + '_' + str(index)
        o_glycan = Sugars(
            assembly_name=name, name=o_glycan_name, source=o_glycan_paths[0],
            protein_name=protein_name, site_indices=[index],
            representation=glycan_representation,
            assembly_params=[0.0, 0.0, 0.0, 0, 0.0, orientation_randon_seed + 1, 0.1])
        be.add_sugars(o_glycan)
            
    '''Complex'''
    indices = [17, 74, 149, 165, 282, 331, 343, 616, 657, 1098, 1134, 1158, 1173, 1194]
    if open_spike:
        indices = [17, 74, 149, 165, 282, 331, 343, 657, 1098, 1134, 1158, 1173, 1194]

    complex_glycans = Sugars(
        rotation=Quaternion(0.0, 0.0, 0.0, 1.0),
        assembly_name=name, name=be.NAME_GLYCAN_COMPLEX, 
        protein_name=name, source=complex_paths[0], 
        site_indices=indices,
        representation=glycan_representation,
        atom_radius_multiplier=glycan_radius_multiplier,
        assembly_params=[0.0, 0.0, 0.0, 0, 0.0, orientation_randon_seed + 2, 0.1]
    )
    status = be.add_glycans(complex_glycans)
            
    '''Materials'''
    be.apply_default_color_scheme(
        shading_mode=be.SHADING_MODE_DIFFUSE,
        specular_exponent=50.0, glossiness=1.0
    )
    
    be.set_models_visibility(True)
    core.set_application_parameters(image_stream_fps=20)

In [ ]:
from IPython.display import clear_output

mm = MovieMaker(be)
output_folder = '/tmp'

for frame in range(1, 20):
    clear_output()
    print('Frame %i' % frame)

    '''Cell'''
    add_spike(frame)
    
    '''Snapshot'''
    mm.create_snapshot(
        size=[1024, 1024], samples_per_pixel=32,
        path=output_folder, base_name='%05d' % frame)